## IPython notebook for the bilateral filter.

### Import all modules and functions needed for this example.

In [ ]:
from skimage.io import imread
from skimage import img_as_float
from skimage.util import random_noise
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm


### Implementation of the bilateral filter.

In [ ]:
# Adapted from bfltGray by Douglas Lanman
# http://www.mathworks.com/matlabcentral/fileexchange/12191-bilateral-filtering/content/Bilateral%20Filtering/bfilter2.m
def bilateralFilter(inputImage, sigma_1, sigma_2, a):

    m, n = inputImage.shape

    # Compute the Gauss kernel for the spatial distance.
    # No need to normalize, this is done when computing
    # the kernel for the intensity distance.
    indX, indY = np.meshgrid(range(-a, a + 1), range(-a, a + 1))
    G_1 = np.exp(-(np.square(indX) + np.square(indY)) / (2 * sigma_1**2))

    outputImage = np.zeros((m, n))
    # tqdm gives us a progress bar, so we see how long the computation of the filter takes.
    tqdm.monitor_interval = 0
    for i in tqdm(range(m), unit="rows"):
        for j in range(n):
            startX = max(0, i - a)
            stopX = min(m, i + a + 1)
            startY = max(1, j - a)
            stopY = min(n, j + a + 1)
            subImage = inputImage[startX:stopX, startY:stopY]

            # Compute the Gauss kernel for the intensity distance.
            G_2 = np.exp(-np.square(subImage - inputImage[i, j]) / (2 * sigma_2**2))

            # Pointwise product of both kernels
            G = np.multiply(G_2, G_1[startX - i + a : stopX - i + a, startY - j + a : stopY - j + a])
            # Finally apply the local kernel and normalize.
            outputImage[i, j] = np.dot(G.ravel(), subImage.ravel()) / np.sum(G)

    return outputImage


### Read and plot the input image.

In [ ]:
true_input_image = img_as_float(imread("astronaut.png", as_gray=True))

plt.imshow(true_input_image, interpolation="nearest", cmap=plt.cm.get_cmap("gray"), vmin=0, vmax=1)
plt.axis("off")
plt.title("Original image")
plt.show()

### Add noise to the input image.

In [ ]:
# Different noise models have very different effects
# input_image = random_noise(true_input_image, mode="gaussian")
input_image = random_noise(true_input_image, mode="poisson")
# input_image = random_noise(true_input_image, mode="s&p")

plt.imshow(input_image, interpolation="nearest", cmap=plt.cm.get_cmap("gray"), vmin=0, vmax=1)
plt.axis("off")
plt.title("Noisy input image")
plt.show()

### Set the filter parameters.

In [ ]:
a = 9
sigma_1 = 2
sigma_2 = 0.1

### Apply the bilateral filter to the noisy input image.

In [ ]:
filtered_image = bilateralFilter(input_image, sigma_1, sigma_2, a)

plt.imshow(filtered_image, interpolation="nearest", cmap=plt.cm.get_cmap("gray"), vmin=0, vmax=1)
plt.axis("off")
plt.title("Bilateral filter")
plt.show()
